<a href="https://colab.research.google.com/github/dlauc/inexactDates/blob/master/extract/InexactDatesGrammar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gramatika nepreciznih datuma
## Cilj: generirati većinu temporalnih izraza koji se često javljaju u wikipediji 
Razlog: pripremiti dataset za učenje pretvaranja tekstualnog datuma u strukturirani oblik


In [4]:
from nltk.parse.generate import generate
from nltk import ChartParser, FeatureEarleyChartParser
from nltk.grammar import CFG, FeatureGrammar
import pandas as pd
#!pip install num2words



In [5]:
#!pip install num2words
from num2words import num2words
# pomoćni
def int_to_roman(input):
    if not isinstance(input, type(1)):
        raise(TypeError, "expected integer, got %s" % type(input))
    if not 0 < input < 4000:
        raise(ValueError, "Argument must be between 1 and 3999")
    ints = (1000, 900,  500, 400, 100,  90, 50,  40, 10,  9,   5,  4,   1)
    nums = ('M',  'CM', 'D', 'CD','C', 'XC','L','XL','X','IX','V','IV','I')
    result = []
    for i in range(len(ints)):
        count = int(input / ints[i])
        result.append(nums[i] * count)
        input -= ints[i] * count
    return ''.join(result)

# test
br = [15,4]
for i in br:
  print(int_to_roman(i), num2words(i,to='ordinal', lang='en'), num2words(i,to='ordinal_num', lang='en'), num2words(i,to='year', lang='en'), num2words(i,to='cardinal', lang='en'))


XV fifteenth 15th fifteen fifteen
IV fourth 4th four four


In [7]:
# Gramatika za stoljeća

STOLJG = """
S[NORM=?n,QUAL=?q] -> STOLJ[NORM=?n,QUAL=?q] 
CENT -> 'century'|'cent.'|'c.'
AD -> 'AD'|'CE'|'A.D.'|'C.E.'
BC -> 'BC'|'BCE'|'B.C.'|'B.C.E.'
STOLJ_N -> CENT | CENT AD
STOLJ_O -> CENT BC
STOLJ_P[QUAL='start'] -> 'the' 'early' | 'the start of the' | 'the start of' | 'start of the' | 'start of' | 'the beginning of the' | 'the beginning of' | 'beginning of the' | 'beginning of' | 'beginning in' | 'beginning in the' | 'early' | 'the early' | 'early in the' | 'early in' | 'the early years of the' | 'the early years in the' | 'early years of the' | 'early years of'
STOLJ_P[QUAL='end'] -> 'the late' | 'late' | 'the end of the' | 'the end of' | 'end of' | 'end of the' | 'later'
STOLJ_P[QUAL='mid'] -> 'the mid-' | 'the middle of the' | 'the mid' | 'mid-' | 'mid'
STOLJ_P[QUAL='fhalf'] -> 'the first half of the' | 'first half of the' | 'first half of'
STOLJ_P[QUAL='shalf'] -> 'the second half' | 'the second half of the' | 'the letter half of the' | 'the letter half'
STOLJ_P[QUAL='dec'] -> 'the early decades of the' | 'the early decades of' | 'early decades of the' | 'early decades of' | 'the last decades of the' | 'the last decades of' | 'last decades of the' | 'last decades of' | 'the first two decades of the' | 'the first two decades of' | 'first two decades of' | 'first two decades of the' | 'the last two decades of the' | 'the last two decades of' | 'last two decades of the' | 'last two decades of' | 'first treee decades of the' | 'first tree decades of' | 'the first tree decades of the' | 'the first tree decades of' | 'the last tree decades of the' | 'the last tree decades of' | 'last tree decades of the' | 'last tree decades of'
STOLJ_P[QUAL='quart'] -> 'the last quarter of the' | 'the last quarter of' | 'last quarter of the' | 'last quarter of' | 'the first quarter of the' | 'the first quarter of' | 'first quarter of the' | 'first quarter of'
STOLJ[NORM=?n,QUAL='0'] -> 'the' STOLJ_S[NORM=?n] STOLJ_N | STOLJ[NORM=?n] STOLJ_N
STOLJ[NORM=?n,QUAL=?q] -> STOLJ_P[QUAL=?q] STOLJ_S[NORM=?n] STOLJ_N
STOLJ[NORM=?n,QUAL=?q] -> STOLJ_P[QUAL=?q] STOLJ_PNE[NORM=?n] STOLJ_O
"""

for i in range(1,25):
  format_str = (i-1, int_to_roman(i),  int_to_roman(i),  int_to_roman(i)+num2words(i,to='ordinal', lang='en')[-2:], 
                i,i,num2words(i,to='ordinal_num', lang='en'), num2words(i,to='cardinal', lang='en'), num2words(i,to='ordinal', lang='en'))
  p = "STOLJ_S[NORM='{:02d}xx'] -> '{}.'|'{}'|'{}'|'{}.'|'{}'|'{}'|'{}'|'{}'".format(*format_str)
  STOLJG += p+'\n'
  #print(p)

for i in range(-50,-1):
  format_str = (-i-1, int_to_roman(-i),  int_to_roman(-i),  int_to_roman(-i)+num2words(-i,to='ordinal', lang='en')[-2:], 
                -i,-i,num2words(-i,to='ordinal_num', lang='en'), num2words(-i,to='cardinal', lang='en'), num2words(-i,to='ordinal', lang='en'))
  p = "STOLJ_PNE[NORM='-{:02d}xx'] -> '{}.'|'{}'|'{}'|'{}.'|'{}'|'{}'|'{}'|'{}'".format(*format_str)
  STOLJG += p+'\n'
  #print(p)
    
gname = 'idg.cfg'
with open('idg.cfg','w') as fo:
    fo.write(STOLJG)
!git add $gname

# generiranje
grammar = FeatureGrammar.fromstring(STOLJG)
for sentence in generate(grammar, n=100):
    print(' '.join(sentence))


the I. century
the I. cent.
the I. c.
the I. century AD
the I. century CE
the I. century A.D.
the I. century C.E.
the I. cent. AD
the I. cent. CE
the I. cent. A.D.
the I. cent. C.E.
the I. c. AD
the I. c. CE
the I. c. A.D.
the I. c. C.E.
the I century
the I cent.
the I c.
the I century AD
the I century CE
the I century A.D.
the I century C.E.
the I cent. AD
the I cent. CE
the I cent. A.D.
the I cent. C.E.
the I c. AD
the I c. CE
the I c. A.D.
the I c. C.E.
the Ist century
the Ist cent.
the Ist c.
the Ist century AD
the Ist century CE
the Ist century A.D.
the Ist century C.E.
the Ist cent. AD
the Ist cent. CE
the Ist cent. A.D.
the Ist cent. C.E.
the Ist c. AD
the Ist c. CE
the Ist c. A.D.
the Ist c. C.E.
the 1. century
the 1. cent.
the 1. c.
the 1. century AD
the 1. century CE
the 1. century A.D.
the 1. century C.E.
the 1. cent. AD
the 1. cent. CE
the 1. cent. A.D.
the 1. cent. C.E.
the 1. c. AD
the 1. c. CE
the 1. c. A.D.
the 1. c. C.E.
the 1 century
the 1 cent.
the 1 c.
the 1 century

In [0]:
# spominjanje stoljeća u wikiju
df1 = pd.read_csv('https://github.com/dlauc/inexactDates/raw/master/stoljeca_wiki.csv')
print('različitih oblika = {}, ukupno = {}'.format(df1.text.count(), df1.n.sum()))


različitih oblika = 3163, ukupno = 97519


In [0]:
# prepoznavanje test
parser = FeatureEarleyChartParser(grammar)
test = df1[df1.n>100]
test = test.sort_values(by=['n'], ascending=False).reset_index(drop=True)

#print(test[0:100])
for i in range(0,100):
  s = test.at[i,'text'].split()
  try:
      trees = list(parser.parse(s))
      if len(trees) == 0:
        print(s, 'neprepoznato')
      else:
        print(s, 'OK')
  except Exception as e:
    print(e)

NameError: ignored

In [0]:
#
